In [1]:
# !apt-get update && apt-get install -y python3-opencv

In [2]:
# !pip install opencv-python

In [3]:
# !python3 -c "import cv2; print(cv2.__version__)"

In [4]:
# !pip install pandas

In [5]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# import matplotlib.pyplot as plt
# from pylab import rcParams

# import torchvision.transforms.functional as TF
# from imgaug import augmentables
# import imgaug as ia
# import imgaug.augmenters as iaa

In [6]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [7]:
import torchvision.models as models
# vgg19 = models.vgg19_bn(pretrained=False)

In [8]:
RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [9]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 45
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EARLY_STOPPING_PATIENCE = 15
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# !python3 -c "import torchvision; print(torchvision.__version__)"

In [11]:
# !pip install torchvision.models.mobilenetv2

In [12]:
# densenet121 = torch.hub.load('pytorch/vision:v0.8.2', 'densenet121', pretrained=False) #161

In [13]:
# densenet121

In [14]:
# densenet121.features[:5]

In [15]:
# densenet121.features[4].denselayer2.norm1

In [16]:
# class Densenet121(nn.Module):
#     def __init__(self, embedding_dimension=128):
#         super().__init__()
#         self.model = densenet121.features[:4]
#         # self.model = densenet121
#         self.layer1 = densenet121.features[4].denselayer1
#         self.dropout1 = nn.Dropout(0.9)
#         self.layer2 = densenet121.features[4].denselayer2
#         self.layer3 = densenet121.features[4].denselayer3
#         self.softmax = nn.Softmax(dim=1)
#         # embedding
#         # input_features_fc_layer = densenet121.classifier.in_features # fc layer 채널 수 얻기
#         # self.classifier = nn.Linear(32768, 2, bias=True) # fc layer 수정
#         # print(self.model)
#         self.classifier = nn.Sequential(
#             nn.Linear(32768 ,512),
#             nn.ReLU(),
#             nn.Dropout(0.8),
#             nn.Linear(512,128),
#             nn.ReLU(),
#             nn.Dropout(0.8),
#             nn.Linear(128,2)
#         )
#     def forward(self, x):
#         # x = self.model(x).view(-1,  16 * 16 * 128) # embedding 생성
#         x1 = self.model(x)
#         x2 = self.layer1(x1)
#         y = torch.cat((x1, x2), 1)
#         x3 = self.layer2(y)
#         y2 = torch.cat((x1, x2, x3), 1)
#         x = self.layer3(y2)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         output = self.softmax(x)

#         return output
#         # return x

In [17]:
# Debug = Densenet121()
# input = torch.Tensor(32, 3, 128, 128)
# Debug(input).shape

In [18]:
resnet18 = torch.hub.load('pytorch/vision:v0.8.2', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.2


In [19]:
# resnet18

In [20]:
resnet18.fc

Linear(in_features=512, out_features=1000, bias=True)

In [21]:
class Resnet18(nn.Module):
    def __init__(self, embedding_dimension=128):
        super().__init__()
        self.model = resnet18
        self.softmax = nn.Softmax(dim=1)
        
        # self.conv1 = self.model.conv1
        # self.bn1 = self.model.bn1
        # self.relu = self.model.relu 
        # self.maxpool = self.model.maxpool
        # self.layer1 = self.model.layer1
        
        # self.model.fc = nn.Linear(65536, 2, bias=True)
        
        # self.fc = nn.Linear(65536, 2, bias=True) # fc layer 수정
        self.fc = nn.Sequential(
            nn.Linear(1000 ,500),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(500,100),
            nn.ReLU(),
            # nn.Dropout(0.8),
            nn.Linear(100,20),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(20,2)
        )
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        output = self.softmax(x)

        return output
        # return x

In [22]:
Debug = Resnet18()
input = torch.Tensor(32, 3, 128, 128)
Debug(input).shape

torch.Size([32, 2])

In [23]:
import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
        
#         return output

In [24]:
# Debug = custom_CNN(2)
# input = torch.Tensor(2, 3, 128, 128)
# Debug(input).shape

In [25]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.RandomRotation(30),
                                             # transforms.RandomHorizontalFlip(),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        
        # tmp = data['COVID']
        # concat_data = pd.concat([tmp, tmp, tmp, tmp])
        return trans_image, data['COVID']

In [26]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

In [27]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

In [28]:
# !pip install sklearn

In [29]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

In [30]:
# !pip install torchvision

In [31]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


In [32]:
# Load Model
model = Resnet18().to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0005, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [33]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)
    
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')
        


  0% 0/45 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.759371260801951, Acc: 0.45611015490533563, F1-Macro: 0.32122153209109733


  2% 1/45 [01:01<45:09, 61.59s/it]

Epoch 0, Val loss: 0.9811893105506897, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Epoch 1, Train loss: 0.7482860154575772, Acc: 0.46815834767641995, F1-Macro: 0.3693235111237577


  4% 2/45 [02:00<43:30, 60.70s/it]

...

 96% 43/45 [34:45<01:30, 45.10s/it]

Epoch 43, Train loss: 0.3470737826493051, Acc: 0.8657487091222031, F1-Macro: 0.865729218516675
Epoch 43, Val loss: 1.0487292408943176, Acc: 0.7384615384615385, F1-Macro: 0.7374673319078165
Early stopping counter 5/15


 98% 44/45 [35:30<00:45, 45.30s/it]

Epoch 44, Train loss: 0.3731638292471568, Acc: 0.846815834767642, F1-Macro: 0.8465958816531534
Epoch 44, Val loss: 0.9456693232059479, Acc: 0.8153846153846154, F1-Macro: 0.8149905123339658
Early stopping counter 6/15


100% 45/45 [36:17<00:00, 48.38s/it]


In [34]:
TRAINED_MODEL_PATH = 'best.pt'

In [35]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [36]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


In [37]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.01s/it]

tensor([[9.7893e-01, 2.1071e-02],
        [3.2508e-01, 6.7492e-01],
        [2.7105e-01, 7.2895e-01],
        [7.0985e-01, 2.9015e-01],
        [7.3383e-01, 2.6617e-01],
        [1.5087e-01, 8.4913e-01],
        [3.9055e-01, 6.0945e-01],
        [2.0742e-01, 7.9258e-01],
        [8.8456e-01, 1.1544e-01],
        [1.2427e-03, 9.9876e-01],
        [9.6019e-01, 3.9814e-02],
        [9.4739e-01, 5.2608e-02],
        [9.8818e-01, 1.1816e-02],
        [9.9884e-01, 1.1584e-03],
        [6.0097e-01, 3.9903e-01],
        [8.4958e-01, 1.5042e-01],
        [3.8673e-01, 6.1327e-01],
        [9.9932e-01, 6.7980e-04],
        [2.1473e-01, 7.8527e-01],
        [9.9177e-01, 8.2292e-03],
        [9.9798e-01, 2.0186e-03],
        [9.6569e-01, 3.4312e-02],
        [6.4185e-01, 3.5815e-01],
        [5.7984e-01, 4.2016e-01],
        [6.4879e-02, 9.3512e-01],
        [9.3990e-01, 6.0098e-02],
        [3.3832e-01, 6.6168e-01],
        [6.9132e-01, 3.0868e-01],
        [5.4692e-03, 9.9453e-01],
        [8.179

2it [00:04,  2.12s/it]

tensor([[0.9310, 0.0690],
        [0.6219, 0.3781],
        [0.6752, 0.3248],
        [0.4777, 0.5223],
        [0.2549, 0.7451],
        [0.2557, 0.7443],
        [0.0856, 0.9144],
        [0.2613, 0.7387],
        [0.3142, 0.6858],
        [0.5825, 0.4175],
        [0.9210, 0.0790],
        [0.0160, 0.9840],
        [0.0568, 0.9432],
        [0.2560, 0.7440],
        [0.7500, 0.2500],
        [0.9951, 0.0049],
        [0.2601, 0.7399],
        [0.9311, 0.0689],
        [0.8452, 0.1548],
        [0.0556, 0.9444],
        [0.1351, 0.8649],
        [0.4023, 0.5977],
        [0.3280, 0.6720],
        [0.0777, 0.9223],
        [0.9151, 0.0849],
        [0.1934, 0.8066],
        [0.9282, 0.0718],
        [0.3738, 0.6262],
        [0.0452, 0.9548],
        [0.9293, 0.0707],
        [0.9940, 0.0060],
        [0.9503, 0.0497]], device='cuda:0')


3it [00:06,  2.15s/it]

tensor([[3.4771e-01, 6.5229e-01],
        [3.0911e-03, 9.9691e-01],
        [1.7689e-01, 8.2311e-01],
        [9.2677e-01, 7.3235e-02],
        [1.9732e-02, 9.8027e-01],
        [7.2010e-01, 2.7990e-01],
        [9.8859e-01, 1.1408e-02],
        [9.6082e-01, 3.9183e-02],
        [3.1866e-01, 6.8134e-01],
        [9.9994e-01, 6.3411e-05],
        [6.7151e-01, 3.2849e-01],
        [9.9561e-01, 4.3885e-03],
        [9.9957e-01, 4.2590e-04],
        [9.9995e-01, 4.5243e-05],
        [1.4820e-01, 8.5180e-01],
        [1.6186e-01, 8.3814e-01],
        [2.4567e-01, 7.5433e-01],
        [1.7081e-01, 8.2919e-01],
        [9.6399e-01, 3.6015e-02],
        [1.7530e-01, 8.2470e-01],
        [3.5253e-01, 6.4747e-01],
        [4.5808e-01, 5.4192e-01],
        [8.5741e-01, 1.4259e-01],
        [7.8840e-02, 9.2116e-01],
        [9.9377e-01, 6.2290e-03],
        [9.0683e-01, 9.3167e-02],
        [9.9012e-01, 9.8796e-03],
        [8.6430e-01, 1.3570e-01],
        [4.4071e-02, 9.5593e-01],
        [4.857

4it [00:06,  1.73s/it]

tensor([[0.1248, 0.8752],
        [0.2135, 0.7865],
        [0.0035, 0.9965],
        [0.4246, 0.5754]], device='cuda:0')


In [38]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)

In [39]:
# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.tolist())
df = pd.DataFrame(pred_lst)
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction111.csv', index=False)

1it [00:02,  2.23s/it]

tensor([[9.7893e-01, 2.1071e-02],
        [3.2508e-01, 6.7492e-01],
        [2.7105e-01, 7.2895e-01],
        [7.0985e-01, 2.9015e-01],
        [7.3383e-01, 2.6617e-01],
        [1.5087e-01, 8.4913e-01],
        [3.9055e-01, 6.0945e-01],
        [2.0742e-01, 7.9258e-01],
        [8.8456e-01, 1.1544e-01],
        [1.2427e-03, 9.9876e-01],
        [9.6019e-01, 3.9814e-02],
        [9.4739e-01, 5.2608e-02],
        [9.8818e-01, 1.1816e-02],
        [9.9884e-01, 1.1584e-03],
        [6.0097e-01, 3.9903e-01],
        [8.4958e-01, 1.5042e-01],
        [3.8673e-01, 6.1327e-01],
        [9.9932e-01, 6.7980e-04],
        [2.1473e-01, 7.8527e-01],
        [9.9177e-01, 8.2292e-03],
        [9.9798e-01, 2.0186e-03],
        [9.6569e-01, 3.4312e-02],
        [6.4185e-01, 3.5815e-01],
        [5.7984e-01, 4.2016e-01],
        [6.4879e-02, 9.3512e-01],
        [9.3990e-01, 6.0098e-02],
        [3.3832e-01, 6.6168e-01],
        [6.9132e-01, 3.0868e-01],
        [5.4692e-03, 9.9453e-01],
        [8.179

2it [00:04,  2.28s/it]

tensor([[0.9310, 0.0690],
        [0.6219, 0.3781],
        [0.6752, 0.3248],
        [0.4777, 0.5223],
        [0.2549, 0.7451],
        [0.2557, 0.7443],
        [0.0856, 0.9144],
        [0.2613, 0.7387],
        [0.3142, 0.6858],
        [0.5825, 0.4175],
        [0.9210, 0.0790],
        [0.0160, 0.9840],
        [0.0568, 0.9432],
        [0.2560, 0.7440],
        [0.7500, 0.2500],
        [0.9951, 0.0049],
        [0.2601, 0.7399],
        [0.9311, 0.0689],
        [0.8452, 0.1548],
        [0.0556, 0.9444],
        [0.1351, 0.8649],
        [0.4023, 0.5977],
        [0.3280, 0.6720],
        [0.0777, 0.9223],
        [0.9151, 0.0849],
        [0.1934, 0.8066],
        [0.9282, 0.0718],
        [0.3738, 0.6262],
        [0.0452, 0.9548],
        [0.9293, 0.0707],
        [0.9940, 0.0060],
        [0.9503, 0.0497]], device='cuda:0')


3it [00:06,  2.23s/it]

tensor([[3.4771e-01, 6.5229e-01],
        [3.0911e-03, 9.9691e-01],
        [1.7689e-01, 8.2311e-01],
        [9.2677e-01, 7.3235e-02],
        [1.9732e-02, 9.8027e-01],
        [7.2010e-01, 2.7990e-01],
        [9.8859e-01, 1.1408e-02],
        [9.6082e-01, 3.9183e-02],
        [3.1866e-01, 6.8134e-01],
        [9.9994e-01, 6.3411e-05],
        [6.7151e-01, 3.2849e-01],
        [9.9561e-01, 4.3885e-03],
        [9.9957e-01, 4.2590e-04],
        [9.9995e-01, 4.5243e-05],
        [1.4820e-01, 8.5180e-01],
        [1.6186e-01, 8.3814e-01],
        [2.4567e-01, 7.5433e-01],
        [1.7081e-01, 8.2919e-01],
        [9.6399e-01, 3.6015e-02],
        [1.7530e-01, 8.2470e-01],
        [3.5253e-01, 6.4747e-01],
        [4.5808e-01, 5.4192e-01],
        [8.5741e-01, 1.4259e-01],
        [7.8840e-02, 9.2116e-01],
        [9.9377e-01, 6.2290e-03],
        [9.0683e-01, 9.3167e-02],
        [9.9012e-01, 9.8796e-03],
        [8.6430e-01, 1.3570e-01],
        [4.4071e-02, 9.5593e-01],
        [4.857

4it [00:07,  1.75s/it]

tensor([[0.1248, 0.8752],
        [0.2135, 0.7865],
        [0.0035, 0.9965],
        [0.4246, 0.5754]], device='cuda:0')
